In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException



import pandas as pd
import time
import re

In [2]:
driver = webdriver.Chrome()

#개발, 프로그래밍 유료, 온라인 강의 페이지 열기
driver.get('https://www.inflearn.com/courses/it?charge=paid&order=seq&types=ONLINE')

In [ ]:
list2 = []
wait = WebDriverWait(driver, .5)
first_iteration = True

continue_loops = True  # 무한반복 방지

while continue_loops:
    try:
        for i in range(1, 25): 
            title_xp = '//*[@id="courses_section"]/div/div/div/main/div[4]/div/div[{}]/div/a/div[2]/div[1]'.format(i)
                
                
            try:
                title = driver.find_element(By.XPATH, title_xp).text
                # 상세페이지 들어가기
                title_element = wait.until(
                    EC.element_to_be_clickable((By.XPATH, title_xp))
                )
                driver.execute_script("arguments[0].click();", title_element)
                
                try:
                    # 총 수강평 수 구하기
                    text = driver.find_element(By.XPATH, '//*[@id="main"]/section/div[1]/div[1]/div/div/div[2]/div[3]/a').text
                
                    #숫자만 추출
                    numbers = re.findall(r'\d+', text)
                    number_review = ''.join(numbers)
                    number_review = int(number_review)
                
                    # 총 수강평 수로 버튼 누를 횟수 정하기
                    if number_review % 5 == 0:
                        more = (number_review // 5) - 1
                    else:
                        more = (number_review // 5)
                        
                    new = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="reviews"]/div[4]/div[1]/span[4]')))
                    # JavaScript를 사용하여 클릭
                    driver.execute_script("arguments[0].click();", new)
        
                    time.sleep(1)

                    # 리뷰 더보기 버튼 누르기
                    for h in range(more):
                        try:
                            # 리뷰 버튼의 XPath
                            review_button_xpath = '//*[@id="reviews"]/div[5]/button'
                            # 리뷰 버튼이 있는 위치로 스크롤 이동
                            review_button = driver.find_element(By.XPATH, review_button_xpath)
                            driver.execute_script("arguments[0].scrollIntoView();", review_button)
                            # JavaScript를 사용하여 클릭
                            driver.execute_script("arguments[0].click();", review_button)

                        
                        except ElementClickInterceptedException: 
                            break
                            
            
                      # 수강평
                    for j in range(1, number_review+1):
                        #수강생의 전체 별점 평균
                        a = '//*[@id="reviews"]/div[4]/div[2]/div[{}]/div[1]/div/div[1]/div/div/span[3]/span'.format(j)
                        star_average = driver.find_element(By.XPATH, a).text
                        
                        #수강생의 해당 강의 별점
                        p = '//*[@id="reviews"]/div[4]/div[2]/div[{}]/div[2]/span[2]'.format(j)
                        star = driver.find_element(By.XPATH, p).text

                        # 리스트 만들기
                        list2.append([title, star_average, star])
                    
                    driver.back()
                            
                except:
                    print(i,title, '강의평이 5개 미만입니다.')
                    # 뒤로 가기
                    driver.back()

            except:
                print('강의가 존재하지 않습니다')
                break
        
        #첫번째 페이지일때 다음페이지로 클릭
        if first_iteration:
            first_page = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="courses_section"]/div/div/div/footer/nav/div/a')))
            # JavaScript를 사용하여 클릭
            driver.execute_script("arguments[0].click();", first_page)
            first_iteration = False
           
        #두번째 페이지부터 다음페이지로 클릭
        else:
            next_page_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="courses_section"]/div/div/div/footer/nav/div/a[2]')))
            driver.execute_script("arguments[0].click();", next_page_button)
            
    except TimeoutException:
        print('페이지가 존재하지 않습니다')
        break



In [ ]:
list2

In [ ]:
#리스트를 데이터프레임으로 변환
df2 = pd.DataFrame(list2, columns=[ '강의명', '총 평균평점', '해당강의 평점'])

In [ ]:
#파일로 저장
df2.to_excel('강의평테스트.xlsx', index=False)